In [1]:
from DcwAggregation import *

verbose = True
extraVerbose = False
if verbose:
    import pprint
    # from IPython.core.display import display
    pprinter = pprint.PrettyPrinter(indent=4)

# get_ipython().magic('matplotlib inline')

# ## Globally relevant variables

saveIdentifiedLineDetails = True  # Need to do this to recompute for every codebook
applyDoubleLineFix = False
applyDoubleWordFilter = True
applyDoubleLineFilter = True
classificationBaseDirectory = 'C:/Users/danny/Repos/text_segmentation/consensus/classifications/'
consensusBaseDirectory = 'C:/Users/danny/Repos/text_segmentation/consensus/consensus/'
databaseNamePattern = 'dcwConsensus_{mss_label}'
aggregatedDataFileNamePattern = 'decoding-the-civil-war-aggregated_{mss_label}.txt'
# 'decoding-the-civil-war-consensus-linewise.csv'
aggregatedDataCsvFileNamePattern = 'decoding-the-civil-war-consensus-linewise_{mss_label}.csv'
# 'decoding-the-civil-war-consensus-subjectwise.csv'
aggregatedDataSubjectWiseCsvFileNamePattern = 'decoding-the-civil-war-consensus-subjectwise_{mss_label}_withBreaks.csv'
# 'dataWithLineIDs_subset.pkl'
identifiedLineFilePathPattern = 'dataWithLineIDs_subset_{mss_label}.pkl'
liveDate = dateutil.parser.parse("2016-06-20T00:00:00.00Z")

# Processing multiple classificaton files
classificationCsvFiles = glob.glob(
    '{}/*.csv'.format(classificationBaseDirectory))

In [2]:
# ledgerIndex = 30
for sampleDataFileName in classificationCsvFiles:
    print('Processing {}...'.format(sampleDataFileName))
    
    mssLabel = sampleDataFileName.split('/')[-1][len(
        'classification_export_'):-4]
    aggregatedDataFileName = aggregatedDataFileNamePattern.format(
        mss_label=mssLabel)  # 'decoding-the-civil-war-aggregated.txt'
    aggregatedDataCsvFileName = aggregatedDataCsvFileNamePattern.format(
        mss_label=mssLabel)  # 'decoding-the-civil-war-consensus-linewise.csv'
    aggregatedDataSubjectWiseCsvFileName = aggregatedDataSubjectWiseCsvFileNamePattern.format(
        mss_label=mssLabel)  # 'decoding-the-civil-war-consensus-subjectwise.csv'
    identifiedLineFilePath = identifiedLineFilePathPattern.format(
        mss_label=mssLabel)  # 'dataWithLineIDs_subset.pkl'

    subjectDataFileName = 'decoding-the-civil-war-subjects-9-29-17.csv'

    subjectsFrame = loadSubjectData(subjectDataFileName)

    telegrams, nTelegramsParsed = loadTelegrams(sampleDataFileName)
    print('Parsed {} telegrams and stored {}.'.format(
        nTelegramsParsed, len(telegrams)))

    transcriptionLineStats, transcriptionLineDetailsFrame = processLoadedTelegrams(
        telegrams)

    transcriptionLineDetailsFrame = groupTranscriptionsLinewise(
        transcriptionLineDetailsFrame, 40, identifiedLineFilePath, saveIdentifiedLineDetails)

    # This is an intentional no-op
    transcriptionLineDetailsFrame = doubleLineFix(
        transcriptionLineDetailsFrame, applyDoubleLineFix=False)

    # The previous step is time consuming so serialize the processed data at
    # this stage
    if saveIdentifiedLineDetails:
        transcriptionLineDetailsFrame.to_pickle(consensusBaseDirectory + identifiedLineFilePath)

    lineGroupedTranscriptionLineDetails = processSentences(
        transcriptionLineDetailsFrame, subjectsFrame)
    saveAggregatedData(lineGroupedTranscriptionLineDetails,
                       consensusBaseDirectory + aggregatedDataCsvFileName, 
                       consensusBaseDirectory + aggregatedDataSubjectWiseCsvFileName,
                       applyDoubleWordFilter, applyDoubleLineFilter, extraVerbose)
    break

Processing C:/Users/danny/Repos/text_segmentation/consensus/classifications\classification_export_mssEC_02_05_25_17.csv...
Parsed 2177 telegrams and stored 232.
Processed 100 subjects...
Processed 200 subjects...
